In [2]:
# load dataset

import pandas

names = ["age", "workclass", "fnlwgt", "education", "education_num", "marital_status", 
         "occupation", "relationship", "race", "sex", "capital_gain", "capital_loss", 
         "hours_per_week", "native_country", "high_income"]
income = pandas.read_csv("income.csv", header=None, names=names, index_col=False)
print(income.head())

   age          workclass  fnlwgt   education  education_num  \
0   39          State-gov   77516   Bachelors             13   
1   50   Self-emp-not-inc   83311   Bachelors             13   
2   38            Private  215646     HS-grad              9   
3   53            Private  234721        11th              7   
4   28            Private  338409   Bachelors             13   

        marital_status          occupation    relationship    race      sex  \
0        Never-married        Adm-clerical   Not-in-family   White     Male   
1   Married-civ-spouse     Exec-managerial         Husband   White     Male   
2             Divorced   Handlers-cleaners   Not-in-family   White     Male   
3   Married-civ-spouse   Handlers-cleaners         Husband   Black     Male   
4   Married-civ-spouse      Prof-specialty            Wife   Black   Female   

   capital_gain  capital_loss  hours_per_week  native_country high_income  
0          2174             0              40   United-States   

In [3]:
# When we construct a decision tree, we'll need to be able to deal with numbers. 
# Therefore, we will convert the categorical variables in our dataset to numeric variables.
# We use the Categorical.from_array method from Pandas to perform the conversion to numbers.

convert_list = ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 
                'race', 'sex', 'native_country', 'high_income']
for column in convert_list:
    col = pandas.Categorical.from_array(income[column])
    income[column] = col.codes
    print(income[column].head())

0    7
1    6
2    4
3    4
4    4
Name: workclass, dtype: int8
0     9
1     9
2    11
3     1
4     9
Name: education, dtype: int8
0    4
1    2
2    0
3    2
4    2
Name: marital_status, dtype: int8
0     1
1     4
2     6
3     6
4    10
Name: occupation, dtype: int8
0    1
1    0
2    1
3    0
4    5
Name: relationship, dtype: int8
0    4
1    4
2    4
3    2
4    2
Name: race, dtype: int8
0    1
1    1
2    1
3    1
4    0
Name: sex, dtype: int8
0    39
1    39
2    39
3    39
4     5
Name: native_country, dtype: int8
0    0
1    0
2    0
3    0
4    0
Name: high_income, dtype: int8


Every time we split when we construct decision trees, we pick a random sample of features from the data. We then compute the information gain for each feature in our random sample, and pick the one with the highest information gain to split on.

Information Gain:

For a simpler explanation of information gain, we're finding the entropy of each set post-split, weighting it by the number of items in each split, then subtracting from the current entropy. If it's positive, we've lowered entropy with our split. The higher it is, the more we've lowered entropy. 

Since Our goal is to ensure that we can make a prediction on future data. In order to do this, all rows in each leaf must have only one value for our target column. If this doesn't happen, we won't be able to make effective predictions. And each split should be closer to the final goal.

Therefore we are going to select the variable which lowers the entropy most.

In [4]:
# Let's first get more familiar with the concept of entropy and information gain.
# We will compute the entropy of the high_income column in the income dataframe 
import math

high_income = sum(income['high_income'] == 1)
total = income.shape[0]
high_ratio = high_income / total
low_ratio = 1 - high_ratio
income_entropy = - (high_ratio * math.log(high_ratio, 2) + low_ratio * math.log(low_ratio, 2))

print(income_entropy)

0.796383955202


In [5]:
# And we will also compute the information gain for splitting on the age column of income
import numpy

# Calculate entropy given a pandas Series, list, or numpy array.
def calc_entropy(column):
    
    counts = numpy.bincount(column)
    probabilities = counts / len(column)
    
    entropy = 0
    for prob in probabilities:
        if prob > 0: 
            entropy += prob * math.log(prob, 2)
    return -entropy

median_age = numpy.median(income['age'])
left_split = income[income['age'] <= median_age]
right_split = income[income['age'] > median_age]

left_entropy = calc_entropy(left_split['high_income'])
right_entropy = calc_entropy(right_split['high_income'])
total_entropy = calc_entropy(income['high_income'])

age_information_gain = total_entropy - (len(left_split) / len(income) * left_entropy + 
                                        len(right_split) / len(income) * right_entropy)
print(age_information_gain)

0.0470286613047


Make a list called information_gains. It should contain, in order, the information gain from splitting on these columns: age, workclass, education_num, marital_status, occupation, relationship, race, sex, hours_per_week, native_country.

Find the highest value in the information_gains list. Assign the name of the column with the highest information gain to highest_gain.

In [6]:
# Calculate information gain given a dataset, column to split on, and the target column.

def calc_information_gain(data_set, split_name, target_name):

    median = numpy.median(data_set[split_name])
    left_split = data_set[data_set[split_name] <= median]
    right_split = data_set[data_set[split_name] > median]
    
    left_entropy = calc_entropy(left_split[target_name])
    right_entropy = calc_entropy(right_split[target_name])
    total_entropy = calc_entropy(data_set[target_name])
    
    information_gain = total_entropy - (len(left_split) / len(data_set) * left_entropy +
                                       len(right_split) / len(data_set) * right_entropy)
    return information_gain

columns = ["age", "workclass", "education_num", "marital_status", "occupation", 
           "relationship", "race", "sex", "hours_per_week", "native_country"]
information_gains = []

# Calculate information gain for each variable in dataset and find the variable that has the
# largest information gain.
for col in columns:
    information_gains.append(calc_information_gain(income, col, 'high_income'))

index = information_gains.index(max(information_gains))
highest_gain = columns[index]
print(highest_gain)

marital_status


In [7]:
# To wrap up, write a function that returns the name of a column to split the data on

def find_best_column(data, target_name, columns):
    # find the column in columns to split on.
    
    information_gains = []
    for col in columns:
        information_gains.append(calc_information_gain(data, col, target_name))
    index = information_gains.index(max(information_gains))
    return columns[index]

# A list of columns to potentially split income with.
columns = ["age", "workclass", "education_num", "marital_status", "occupation", 
           "relationship", "race", "sex", "hours_per_week", "native_country"]

income_split = find_best_column(income, 'high_income', columns)
print(income_split)

marital_status


After knowing the basics of entropy and information gain, we'll use the ID3 Algorithm to construct full decision trees. This algorithm involves recursion. 

In general, recursion is the process of splitting a large problem into small chunks. Recursive functions will call themselves, then combine the results to create a final result.

Building trees is a perfect case for a recursive algorithm -- at each node, we'll call a recursive function, which will split the data into two branches. Each branch will lead to a node, and the function will call itself to build out the tree.

In [8]:
# To build up to making the full id3 function, first build a simpler algorithm that we can 
# extend. we'll just count up how many leaves end up with the label 1 (>50k), and how many 
# end up with the label 0 (<= 50k).

label_1s = []
label_0s = []
def id3(data_set, target, columns):
    unique_targets = pandas.unique(data_set[target])
    
    if len(unique_targets) == 1:
        if (unique_targets[0] == 1):
            label_1s.append(1)
        elif(unique_targets[0] == 0):
            label_0s.append(0)
        return
    
    best_column = find_best_column(data_set, target, columns)
    column_median = numpy.median(data_set[best_column])
    left_split = data_set[data_set[best_column] <= column_median]
    right_split = data_set[data_set[best_column] > column_median]
    
    for split in [left_split, right_split]:
        id3(split, target, columns)

# Create a model dataset.
data = pandas.DataFrame([
    [0,20,0],
    [0,60,2],
    [0,40,1],
    [1,25,1],
    [1,35,2],
    [1,55,1]
    ])
# Assign column names to the data.
data.columns = ["high_income", "age", "marital_status"]

# Call the function on our data to set the counters.
id3(data, "high_income", ["age", "marital_status"])
print(label_1s, label_0s)

[1, 1, 1] [0, 0, 0]


In [9]:
# Now we'll use dictionaries to store the entire tree instead of just the labels at leaves.  
tree = {}
nodes = []

def id3(data_set, target, columns, tree):
    nodes.append(len(nodes) + 1)
    tree['number'] = nodes[-1]
    
    unique_targets = pandas.unique(data_set[target])
    
    if len(unique_targets) == 1:
        if (unique_targets[0] == 1):
            tree['label'] = 1
        elif(unique_targets[0] == 0):
            tree['label'] = 0
        return
    
    best_column = find_best_column(data_set, target, columns)
    column_median = numpy.median(data_set[best_column])
    
    tree['column'] = best_column
    tree['median'] = column_median
    
    left_split = data_set[data_set[best_column] <= column_median]
    right_split = data_set[data_set[best_column] > column_median]
    
    for name, split in [['left', left_split], ['right', right_split]]:
        tree[name] = {}
        id3(split, target, columns, tree[name])

id3(data, "high_income", ["age", "marital_status"], tree)
print(tree)

{'column': 'age', 'left': {'column': 'age', 'left': {'column': 'age', 'left': {'label': 0, 'number': 4}, 'number': 3, 'median': 22.5, 'right': {'label': 1, 'number': 5}}, 'number': 2, 'median': 25.0, 'right': {'label': 1, 'number': 6}}, 'number': 1, 'median': 37.5, 'right': {'column': 'age', 'left': {'column': 'age', 'left': {'label': 0, 'number': 9}, 'number': 8, 'median': 47.5, 'right': {'label': 1, 'number': 10}}, 'number': 7, 'median': 55.0, 'right': {'label': 0, 'number': 11}}}


In [10]:
# Print out dictionary in a nicer way
def print_with_depth(string, depth):
    # Add space before a string.
    prefix = "    " * depth
    # Print a string, appropriately indented.
    print("{0}{1}".format(prefix, string))
    
    
def print_node(tree, depth):
    # Check for the presence of label in the tree.
    if "label" in tree:
        # If there's a label, then this is a leaf, so print it and return.
        print_with_depth("Leaf: Label {0}".format(tree["label"]), depth)
        # This is critical -- without it, we'll get infinite recursion.
        return
    # Print information about what the node is splitting on.
    print_with_depth("{0} > {1}".format(tree["column"], tree["median"]), depth)
    
    # Create a list of tree branches.
    branches = [tree["left"], tree["right"]]
        
    for branch in branches:
        print_node(branch, depth+1)

print_node(tree, 0)

age > 37.5
    age > 25.0
        age > 22.5
            Leaf: Label 0
            Leaf: Label 1
        Leaf: Label 1
    age > 55.0
        age > 47.5
            Leaf: Label 0
            Leaf: Label 1
        Leaf: Label 0


In [11]:
# Make predictions
def predict(tree, row):
    if 'label' in tree:
        return tree['label']
    
    column = tree['column']
    median = tree['median']
    
    if row[column] <= median:
        return predict(tree['left'], row)
    else:
        return predict(tree['right'], row)

print(predict(tree, data.iloc[0]))

0


In [12]:
# Make predictions on multiple rows at once 
new_data = pandas.DataFrame([
    [40,0],
    [20,2],
    [80,1],
    [15,1],
    [27,2],
    [38,1]
    ])
# Assign column names to the data.
new_data.columns = ["age", "marital_status"]

predictions = new_data.apply(lambda row: predict(tree, row), axis=1)
print(predictions)

0    0
1    0
2    0
3    0
4    1
5    0
dtype: int64


In [13]:
# Use the scikit-learn package to fit a decision tree

from sklearn.tree import DecisionTreeClassifier

columns = ["age", "workclass", "education_num", "marital_status", "occupation", 
           "relationship", "race", "sex", "hours_per_week", "native_country"]

# Set random_state to 1 to keep results consistent.
clf = DecisionTreeClassifier(random_state=1)
clf.fit(income[columns], income['high_income'])

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=1, splitter='best')

We can split the data by shuffling the order of the dataframe, then selecting certain rows to be in the training set, and certain rows to be in the testing set. Here we'll make 70% of our rows training data, and 30% testing data.

In [14]:
import numpy
import math

# Set a random seed so the shuffle is the same every time.
numpy.random.seed(1)

# Shuffle the rows.

income = income.reindex(numpy.random.permutation(income.index))

train_max_row = math.floor(income.shape[0] * .7)

train = income.iloc[:train_max_row]
test = income.iloc[train_max_row:]

While there are many methods to evaluate error with classification, we'll use AUC here. AUC ranges from 0 to 1, and is ideal for binary classification. The higher the AUC, the more accurate our predictions.

In [15]:
# Evaluate error using AUC
from sklearn.metrics import roc_auc_score

clf = DecisionTreeClassifier(random_state=1)
clf.fit(train[columns], train["high_income"])

predictions = clf.predict(test[columns])

error = roc_auc_score(predictions, test['high_income'])
print(error)

0.70548884174


In [16]:
# Compute error on the training set
predictions = clf.predict(train[columns])
error = roc_auc_score(predictions, train['high_income'])
print(error)

0.976351819595


AUC on the training set was 0.976. The AUC on the test set was 0.705. Our model is predicting the training set much better than it's predicting the test set. Based on our AUC measurements, it appears that we are in fact overfitting.

In [17]:
# To reduce overfitting, adjust min_samples_split: the minimum number of rows needed in a node 
# before it can be split. 
# Set min_samples_split to 5

clf = DecisionTreeClassifier(random_state=1, min_samples_split=5)
clf.fit(train[columns], train['high_income'])

predictions_train = clf.predict(train[columns])
train_auc = roc_auc_score(predictions_train, train['high_income'])

predictions_test = clf.predict(test[columns])
test_auc = roc_auc_score(predictions_test, test['high_income'])

print(train_auc)
print(test_auc)

0.935645711968
0.708880907772


By restricting min_samples_split to 5, we increased test AUC to 0.709 from 0.705. Training set AUC decreased from 0.976 to 0.936, showing that the model we built was less overfit to the training set than before.

In [18]:
# More parameter tweaking: 
# Adjust max_depth, which restricts how deep the tree can go.

clf = DecisionTreeClassifier(random_state=1, min_samples_split=25, max_depth=4)
clf.fit(train[columns], train["high_income"])
predictions = clf.predict(test[columns])
test_auc = roc_auc_score(predictions, test["high_income"])

train_predictions = clf.predict(train[columns])
train_auc = roc_auc_score(train_predictions, train["high_income"])

print(test_auc)
print(train_auc)

0.79777759783
0.792384918107


As both AUCs are about the same, we aren't overfitting anymore.